In [1]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

CONTENT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'vi_core_news_lg')
STATUS = data.LabelField(dtype = torch.float)

In [24]:
fields = [('content', CONTENT), ('status', STATUS)]

train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = './input',
                                        train = 'train_filtered.csv',
                                        validation = 'validation_filtered.csv',
                                        test = 'test_filtered.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True
)

In [25]:
print(vars(train_data.examples[1]))

{'content': ['giữ', 'vững', 'top', 'nào', 'các', 'đóm'], 'status': '1'}


In [26]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validating examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 870391
Number of validating examples: 62125
Number of testing examples: 47755


In [27]:
print(vars(train_data.examples[0]))

{'content': ['Nam_Mô', 'A_Di_Đà_Phật'], 'status': '1'}


In [28]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [29]:
MAX_VOCAB_SIZE = 25_000

CONTENT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
STATUS.build_vocab(train_data)

print(f"Unique tokens in CONTENT vocabulary: {len(CONTENT.vocab)}")
print(f"Unique tokens in STATUS vocabulary: {len(STATUS.vocab)}")

Unique tokens in CONTENT vocabulary: 25002
Unique tokens in STATUS vocabulary: 2


In [30]:
print(CONTENT.vocab.freqs.most_common(20))

[('hay', 175021), ('quá', 126783), ('nghe', 62082), ('anh', 57462), ('không', 53718), ('bài', 50228), ('hát', 49351), ('ơi', 43399), ('này', 42938), ('là', 38455), ('đi', 37055), ('em', 35994), ('luôn', 29155), ('có', 28578), ('mà', 28169), ('Hay', 26981), ('của', 26211), ('chị', 22377), ('rồi', 21962), ('nhạc', 21457)]


In [31]:
print(CONTENT.vocab.itos[:10])

['<unk>', '<pad>', 'hay', 'quá', 'nghe', 'anh', 'không', 'bài', 'hát', 'ơi']


In [32]:
print(STATUS.vocab.stoi)

defaultdict(None, {'1': 0, '0': 1})


In [33]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.content),
    sort_within_batch = False,
    device = device)

In [34]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [35]:
INPUT_DIM = len(CONTENT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [36]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [37]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [38]:
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [39]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [40]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.content).squeeze(1)
        
        loss = criterion(predictions, batch.status)
        
        acc = binary_accuracy(predictions, batch.status)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [41]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.content).squeeze(1)
            
            loss = criterion(predictions, batch.status)
            
            acc = binary_accuracy(predictions, batch.status)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [42]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [43]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './model_rnn/model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 6m 53s
	Train Loss: 0.375 | Train Acc: 87.45%
	 Val. Loss: 0.617 |  Val. Acc: 77.06%
Epoch: 02 | Epoch Time: 6m 50s
	Train Loss: 0.372 | Train Acc: 87.70%
	 Val. Loss: 0.574 |  Val. Acc: 84.63%
Epoch: 03 | Epoch Time: 6m 50s
	Train Loss: 0.372 | Train Acc: 87.79%
	 Val. Loss: 0.540 |  Val. Acc: 86.57%
Epoch: 04 | Epoch Time: 6m 47s
	Train Loss: 0.371 | Train Acc: 87.82%
	 Val. Loss: 0.512 |  Val. Acc: 87.35%
Epoch: 05 | Epoch Time: 7m 6s
	Train Loss: 0.371 | Train Acc: 87.83%
	 Val. Loss: 0.488 |  Val. Acc: 87.61%


In [44]:
model.load_state_dict(torch.load('./model_rnn/model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.508 | Test Acc: 85.91%
